## 논리학습 신경망(오차역전파)

In [83]:
import numpy as np
from datetime import datetime      # datetime.now() 를 이용하여 학습 경과 시간 측정

def sigmoid(x):         # sigmoid 함수
    return 1 / (1+np.exp(-x))

def cross_entropy(t, y) :
    delta = 1e-7    # log 무한대 발산 방지
    return -np.sum(t*np.log(y+delta) + (1-t)*np.log((1-y)+delta))    

class LogicGate:
    def __init__(self, gate_name, xdata, tdata):
        self.name = gate_name
            # 입력 데이터, 정답 데이터 초기화
        self.xdata = xdata.reshape(4,2)  # 4개의 입력데이터 x1, x2 에 대하여 batch 처리 행렬
        self.tdata = tdata.reshape(4,1)  # 4개의 입력데이터 x1, x2 에 대한 각각의 계산 값 행렬
            # 2층 hidden layer unit : 5개 가정,  가중치 W2, 바이어스 b2 초기화
        self.W2 = np.random.rand(2,5)  # weight, 2 X 6 matrix
        self.b2 = np.random.rand(5)
            # 3층 output layer unit : 1 개 , 가중치 W3, 바이어스 b3 초기화
        self.W3 = np.random.rand(5,1)
        self.b3 = np.random.rand(1)     
            # 학습률 learning rate 초기화
        self.lr = 1e-2
        print(self.name + " object is created")
            
    def feed_forward(self):        # feed forward 를 통하여 손실함수(cross-entropy) 값 계산
        self.a1 = self.xdata        # 입력층 출력 = 입력과 동일
        self.z2 = np.dot(self.a1, self.W2) + self.b2     # 은닉층의 선형회귀 값
        self.a2 = sigmoid(self.z2)                          # 은닉층의 출력
        
        self.z3 = np.dot(self.a2, self.W3) + self.b3            # 출력층의 선형회귀 값
        y = self.a3 = sigmoid(self.z3)                          # 출력층의 출력
        return cross_entropy(self.tdata, y)    

    def err_back_propa(self)
        loss_val = self.feed_forward() # 먼저 feed forward 를 통해서 최종 출력값과 이를 바탕으로 현재의 에러 값 계산

        Loss3 = (self.a3-self.tdata) * self.a3 * (1-self.a3)  # 출력층 loss 인 Loss3 구함             
        # 출력층 가중치 W3, 출력층 바이어스 b3 업데이트
        self.W3 = self.W3 - self.lr * np.dot(self.a2.T, Loss3)   
        self.b3 = self.b3 - self.lr * Loss3
        self.b3 = np.mean(self.b3, axis=0).reshape(1,-1) # numpy broadcasting으로 변형된 배열형태를 복귀

        Loss2 = np.dot(Loss3, self.W3.T) * self.a2 * (1-self.a2)  # 은닉층 loss 인 Loss2 구함
        # 은닉층 가중치 W2, 은닉층 바이어스 b2 업데이트
        self.W2 = self.W2 - self.lr * np.dot(self.a1.T, Loss2)   
        self.b2 = self.b2 - self.lr * Loss2
        self.b2 = np.mean(self.b2, axis=0).reshape(1,-1) # numpy broadcasting으로 변형된 배열형태를 복귀
    
    def errValue(self):          # 외부 출력을 위한 손실함수(cross-entropy) 값 계산 
        return self.feed_forward()    
    
    def train(self, learning_rate=1e-2, repeat=10000, outStep=1000):  # 오차역전파 학습
        self.lr = learning_rate
        start = datetime.now()
        for step in range(repeat+1):
            self.err_back_propa()                      # 4개 입력 데이타를 배치 처리
            # for i in range(len(self.xdata)):         # 각 입력마다 역전파 수행
            #     self.err_back_propa(self.xdata[i:i+1, ], self.tdata[i:i+1, ])
            if step % outStep == 0:
                print("step = %6d,"% step,  "loss_val = %.5f"% self.errValue())
        print("Training time = ", datetime.now() - start)

    def predict(self, test):
        for t in test:
            z2 = np.dot(t, self.W2) + self.b2         # 은닉층의 선형회귀 값
            a2 = sigmoid(z2)                                  # 은닉층의 출력
            z3 = np.dot(a2, self.W3) + self.b3            # 출력층의 선형회귀 값
            a3 = sigmoid(z3)                              # 출력층의 출력
            y = a3[0,0]
            print(t, '-->', int(y>0.5), "[{:.3f}]".format(y))

In [90]:
# XOR Gate 객체 생성
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 0])

xor = LogicGate("XOR", xdata, tdata)
xor.train(0.05, 80000, 5000)

XOR object is created
step =      0, loss_val = 4.51689
step =   5000, loss_val = 2.34427
step =  10000, loss_val = 0.62169
step =  15000, loss_val = 0.31567
step =  20000, loss_val = 0.22926
step =  25000, loss_val = 0.18665
step =  30000, loss_val = 0.16055
step =  35000, loss_val = 0.14260
step =  40000, loss_val = 0.12935
step =  45000, loss_val = 0.11907
step =  50000, loss_val = 0.11082
step =  55000, loss_val = 0.10401
step =  60000, loss_val = 0.09827
step =  65000, loss_val = 0.09335
step =  70000, loss_val = 0.08908
step =  75000, loss_val = 0.08532
step =  80000, loss_val = 0.08199
Training time =  0:00:08.176506


In [91]:
test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
print(xor.predict(test_data))

[0 0] --> 0 [0.009]
[0 1] --> 1 [0.978]
[1 0] --> 1 [0.977]
[1 1] --> 0 [0.028]
None


In [92]:
test_data = np.array([ [0, 0.4], [0.2, 0.7], [0.6, 0.3], [0.9, 0.8], [0.1, 0.6] ])
print(xor.predict(test_data))

[0.  0.4] --> 1 [0.514]
[0.2 0.7] --> 1 [0.761]
[0.6 0.3] --> 0 [0.378]
[0.9 0.8] --> 0 [0.039]
[0.1 0.6] --> 1 [0.809]
None
